# Module

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset, MFDataset
import netCDF4
import matplotlib.pyplot as plt
import glob
from datetime import datetime, timedelta
from scipy import stats
import shutil
from scipy.spatial import distance
import metpy
import xskillscore as xs
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
from cartopy.feature import NaturalEarthFeature
from glob import *
import sys, os, time, warnings
import pandas as pd
import pytz
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore')
from dask.diagnostics import ProgressBar
import metpy

# function  
아래의 함수들 포함.  

fft_1st_phase(a)  
find_nearest(array, value)  
find_nearest_idx(array, value) 
harm_1st_max_idx(array, values)  
diurnal_cycle_fig(data,daily_mean_data,time,data_source)  
grid_transfer(data)  
regrid(change_data,target_data)  
amplitude_fig(data,data_source)amplitude_fig(data,data_source)  
print(f"latitude : {i}", end='\r') # 순서 보여주는거


### harmonic analysis function

In [2]:
def fft_1st_phase(a):
    ff = np.fft.fft(a)
    # Get the complex vector at that frequency to retrieve amplitude and phase shift
    yy = ff[1] 

    # Calculate the amplitude
    T = a.shape[0] # domain of x; which we will divide height to get freq amplitude
    A = np.sqrt(yy.real**2 + yy.imag**2)/T 
    # print('amplitude of:', A) 
    
    # Calculate phase shift
    phi = np.arctan(yy.imag/yy.real)
    # print('phase change:', phi)
    
    return phi, yy.real ,A


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def harm_1st_max_idx(array, values): # 앞에가 시간 뒤에가 강수
    pp=0
    if fft_1st_phase(values)[1]>0:
        if fft_1st_phase(values)[0]<0:
            pp = -fft_1st_phase(values)[0]/np.pi *np.pi
        elif fft_1st_phase(values)[0]>0:
            pp = ( 2. - fft_1st_phase(values)[0]/np.pi ) *np.pi
    elif fft_1st_phase(values)[1]<0:
        pp = ( -fft_1st_phase(values)[0]/np.pi + 1. ) *np.pi    
    
    pp2 = np.nanmax(array,axis=0) * pp / (2.*np.pi)    
    return find_nearest_idx(array, pp2)

### diurnal cycle fig function

In [3]:
def diurnal_cycle_fig(data,daily_mean_data,time,data_source):

    if time == 'utc':
        tt = 'UTC(hour)'
    else : 
        tt = 'LST(hour)'
        
    tick_spacing = 30

    projection = ccrs.PlateCarree()
    crs = ccrs.PlateCarree()
    fig = plt.figure(figsize=(20,10))
    ax = plt.axes(projection=projection, frameon=True)


    varMin, varMax, varInt = 0, 24, 1
    levels = np.arange(varMin, varMax+varInt, varInt)
    nlevs  = levels.size
    tick_interval = 2
    cmap = plt.get_cmap('Spectral', nlevs)
    extent=[-180, 180, -60, 60]
    gl = ax.gridlines(crs=crs, draw_labels=True, linewidth=1, color='gray', alpha=1, linestyle='-.')
    gl.xlabel_style = {"size" : 13}
    gl.ylabel_style = {"size" : 13}
    gl.xlocator = mticker.FixedLocator(np.arange(extent[0], extent[1] + tick_spacing, tick_spacing))
    gl.ylocator = mticker.FixedLocator(np.arange(extent[2], extent[3] + tick_spacing, tick_spacing))

    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter()) 

    gl.top_labels = None
    gl.right_labels = None

    ax.add_feature(cfeature.COASTLINE.with_scale("50m"), lw=0.8)
    ax.add_feature(cfeature.OCEAN.with_scale("50m"), edgecolor='none', facecolor='lightgray')
    ax.add_feature(cfeature.BORDERS.with_scale("50m"), lw=0.3)
    ax.set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())

    lons = data['longitude']  # 경도
    lats = data['latitude']  # 위도
    values = data[time]

    lons, lats = np.meshgrid(data['longitude'], data['latitude'])

    cnplot = ax.contourf(lons, lats, values,cmap=cmap,levels=levels,zorder=0,transform=ccrs.PlateCarree())

    cbar = plt.colorbar(cnplot,ticks=np.arange(varMin, varMax+tick_interval, tick_interval), 
                        orientation='vertical', pad=0.01, shrink=.575) 

    zm = np.ma.masked_less(daily_mean_data, 0.275)
    plt.pcolor(lons, lats,zm, hatch='xxx', alpha=0.)

    cbar.set_label(tt, fontsize=15)
    cbar.ax.tick_params(labelsize=15)

    plt.xlabel('Longitude',fontsize=10,fontweight='bold')
    plt.ylabel('Latitude',fontsize=10,fontweight='bold')    
    ax.set_title(data_source+' [JJA 2022]',fontsize=20,fontweight='bold')
    plt.show()


### grid inverse function

In [4]:
def grid_transfer(data):
    data=data.sortby(data['latitude'], ascending=True) #위도 ㅂ반전
    data['longitude'] = xr.where(data['longitude'] > 180, data['longitude'] - 360, data['longitude']) # 경도 반전
    data = data.sortby(data['longitude']) #바뀐 경도에 대해 값들 맞춰줌 한번만 실행할것
    
    return data

### amplitude_fig

In [5]:
def amplitude_fig(data,data_source):

    tick_spacing = 30
    projection = ccrs.PlateCarree()
    crs = ccrs.PlateCarree()
    fig = plt.figure(figsize=(20,10))
    ax = plt.axes(projection=projection, frameon=True)


    varMin, varMax, varInt = 0, 2, 0.1
    levels = np.arange(varMin, varMax+varInt, varInt)
    nlevs  = levels.size
    tick_interval = 0.2
    cmap = plt.get_cmap('plasma', nlevs)
    extent=[-180, 180, -60, 60]

    gl = ax.gridlines(crs=crs, draw_labels=True, linewidth=1, color='gray', alpha=1, linestyle='-.')
    gl.xlabel_style = {"size" : 13}
    gl.ylabel_style = {"size" : 13}
    gl.xlocator = mticker.FixedLocator(np.arange(extent[0], extent[1] + tick_spacing, tick_spacing))
    gl.ylocator = mticker.FixedLocator(np.arange(extent[2], extent[3] + tick_spacing, tick_spacing))

    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter()) 

    gl.top_labels = None
    gl.right_labels = None

    ax.add_feature(cfeature.COASTLINE.with_scale("50m"), lw=0.8)
    ax.add_feature(cfeature.OCEAN.with_scale("50m"), edgecolor='none', facecolor='lightgray')
    ax.add_feature(cfeature.BORDERS.with_scale("50m"), lw=0.3)
    ax.set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())

    lons = data['longitude']  # 경도
    lats = data['latitude']  # 위도
    values = data['amlitude']

    lons, lats = np.meshgrid(data['longitude'], data['latitude'])

    cnplot = ax.contourf(lons, lats, values,cmap=cmap,levels=levels,zorder=0,
                         transform=ccrs.PlateCarree(),extend='max')

    cbar = plt.colorbar(cnplot,ticks=np.arange(varMin, varMax+tick_interval, tick_interval), 
                        orientation='vertical', pad=0.01, shrink=.56) 

    # zm = np.ma.masked_less(daily_mean_data, 0.275)
    # plt.pcolor(lons, lats,zm, hatch='xxx', alpha=0.)

    cbar.set_label('Amplitude', fontsize=15)
    cbar.ax.tick_params(labelsize=15)

    plt.xlabel('Longitude',fontsize=10,fontweight='bold')
    plt.ylabel('Latitude',fontsize=10,fontweight='bold')    
    ax.set_title(data_source+' [JJA 2022]',fontsize=20,fontweight='bold')
    plt.show()


### grid same

In [6]:
import xesmf as xe

In [7]:
def regrid(change_data,target_data):
    grid_frame = xe.Regridder(change_data, target_data, "bilinear")
    regrid_data=grid_frame(change_data)
    return regrid_data

# ERA5 자료 처리

Mean rate/flux parameters in ERA5 (e.g. Table 4 for surface and single levels) provide similar information to accumulations   
(e.g. Table 3 for surface and single levels), except they are expressed as temporal means, over the same processing periods, and so have units of "per second".  

Mean rate/flux parameters are easier to deal with than accumulations because the units do not vary with the processing period.  
The mean rate hydrological parameters (e.g. the "Mean total precipitation rate") have units of "kg m-2 s-1", which are equivalent to "mm s-1".   
They can be multiplied by 86400 seconds (24 hours) to convert to kg m-2 day-1 or mm day-1.  

## Make lst data

In [2]:
year = np.arange(1979,2023)
hour = np.arange(0,24,3)

In [3]:
file_list=sorted(glob('/data1/user/gychoi/Data/ERA_land/ERA_lst_base' + '*.nc'))

In [4]:
file_list

['/data1/user/gychoi/Data/ERA_land/ERA_lst_base1979.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1980.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1981.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1982.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1983.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1984.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1985.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1986.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1987.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1988.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1989.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1990.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1991.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1992.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1993.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1994.nc',
 '/data1/user/gychoi/Data/ERA_land/ERA_lst_base1995.nc',
 '/data1/user/gychoi/Data/ERA_l

In [5]:
hour = np.arange(0,24,3)
file = []

for i in range (len(year[10:10+10])):
    file = []
    data=xr.open_dataset(file_list[10:10+10][i])

    for j in range (len(hour)):
        hourly = data.sel(time = data['time.hour']==hour[j]).resample(time='D').mean('time')
        file.append(hourly.sel(time=(( hourly['time.month'] >= 6) & ( hourly['time.month'] <= 8)),drop=True))

        print(f'hour =',hour[j],end='\r')

    file = xr.concat(file,'hour')
    file['hour'] = hour
    file.to_netcdf('/data1/user/gychoi/Data/ERA_land/LST/ERA_lst'+str(year[10:10+10][i])+'.nc')

    
    del(file)
    data.close()
    
    print(f'year =',year[10:10+10][i],end='\r')